In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn

In [2]:
vocab_size = 16
X = torch.randint(0, vocab_size, (5, ))
X_onehot = F.one_hot(X, num_classes=vocab_size).float()

In [3]:

# X -> Vec -> Y
# we take the sequence X into a vector represetation (Encoder)
# And later we take that vector representation and tranform it in to a new sequence Y (Decoder)


# Encoder

#for i in T:
	#hx[t] = f(hx[t-1], x[t])
#C = hx[T-1]

#Decoder
# hy[t] = f(hy[t-1], y[t-1], C)


# Encoder

#for i in T:
	#hx[t] = f(hx[t-1], x[t])
#C = hx[T-1]

In [3]:
hidden_size = 32
# reset gate weights
Wr = torch.randn(vocab_size, hidden_size)
Ur  = torch.randn(hidden_size, hidden_size)

# update gate weights
Wz = torch.randn(vocab_size, hidden_size)
Uz  = torch.randn(hidden_size, hidden_size)

# x-to-h Weights
Wxh = torch.randn(vocab_size, hidden_size)
Whh = torch.randn(hidden_size, hidden_size)

#
V = torch.randn(hidden_size, hidden_size)


In [4]:
h = torch.zeros((1, hidden_size))
t = 0

r = F.sigmoid((X_onehot[t] @ Wr) + (h @ Ur)) # the reset gate
z = F.sigmoid((X_onehot[t] @ Wr) + (h @ Ur)) # updated gate

# This gate decides if we keep h[t-1] to update h
# if r is rougly zeros, ignores the previous hidden state and only carries x[t]'s projection to h 
# to perform the update.
reset_gate = (X_onehot[t] @ Wxh) + (h * r) @ Whh

# the update gate, decides wheter we need to update h or not, however not binary yes or no.
# is a smoothed decision, interpolates what was kept from the reset gate in [0,1]
h1 = z * h + (1-z) * reset_gate


In [5]:
# Encoder
T, _ = X_onehot.shape

h = torch.zeros((1, hidden_size))

for t in range(T):
	r = F.sigmoid((X_onehot[t] @ Wr) + (h @ Ur)) # the reset gate
	z = F.sigmoid((X_onehot[t] @ Wz) + (h @ Uz)) # updated gate
	update = F.tanh((X_onehot[t] @ Wxh) + (r * h) @ Whh)
	h = z*h + (1-z)*update

C = F.tanh(h@V)

In [6]:
import kagglehub
import pandas as pd

In [7]:
path = kagglehub.dataset_download("lonnieqin/englishspanish-translation-dataset")

100%|██████████| 2.72M/2.72M [00:01<00:00, 2.12MB/s]

Extracting files...


In [8]:
import os
from pathlib import Path
file =  Path(path) / "data.csv"
df = pd.read_csv(file)

In [9]:
df.head()

,english,spanish
0,Go.,Ve.
1,Go.,Vete.
2,Go.,Vaya.
3,Go.,Váyase.
4,Hi.,Hola.


In [10]:
import re

class Tokenizer:
	def __init__(self, target: str):
		if target not in ["english", "spanish"]:
			raise ValueError("Target must be one of : ['english', 'spanish']")

		# special tokens
		self.eos_token = "<EOS>"
		self.sos_token = "<SOS>"
		self.unk_token = "<UNK>"

		#vocab
		self.vocab = self.build_vocab(target, special_tokens=[self.eos_token, self.sos_token])
		self.vocab_size = len(self.vocab)

		# mappings
		self.token_to_idx = {token: idx for idx, token in enumerate(self.vocab)}
		self.idx_to_token = {idx: token for idx, token in enumerate(self.vocab)}

		# mappings for unk token
		self.token_to_idx[self.unk_token] = len(self.vocab)
		self.idx_to_token[len(self.vocab)] = self.unk_token

	def tokenize(self, s: str) -> list[str]:
		s = re.sub(r"([.!?])", r" \1", s)
		tokens = s.split()
		return tokens

	def build_vocab(self, target: str, special_tokens: list[str]):
		vocab = set()
		sentences = df[target]
		for s in sentences:
			tokens = self.tokenize(s)
			for token in tokens:
				vocab.add(token)
		return list(vocab) + special_tokens

	def encode(self, s: str) -> torch.Tensor:
		out = [self.token_to_idx[self.sos_token]]
		for token in self.tokenize(s):
			if token not in self.vocab:
				out.append(self.token_to_idx[self.unk_token])
			else:
				out.append(self.token_to_idx[token])
		return torch.tensor(out + [self.token_to_idx[self.eos_token]])

	def decode(self, indices: list[int]) -> str:
		out = []
		for idx in indices:
			if idx == self.token_to_idx[self.unk_token]:
				out.append(self.unk_token)
			else:
				out.append(self.idx_to_token[idx])
		return " ".join(out)

In [11]:
en_tokenizer = Tokenizer(target="english")
es_tokenizer = Tokenizer(target="spanish")

In [12]:
tokens_en = en_tokenizer.encode("Hello how are you today my friend?")
tokens_es = es_tokenizer.encode("hola como estas hoy amigo? xdxd")

In [13]:
print(tokens_en, en_tokenizer.decode(tokens_en.tolist()))
print(tokens_es, es_tokenizer.decode(tokens_es.tolist()))

tensor([17398,  4531,  3007,  6408, 10345,  4212, 11407,  5517,  2270, 17397]) <SOS> Hello how are you today my friend ? <EOS>
tensor([32338, 29632,  8981, 12631, 24980, 13585,  9146, 32339, 32337]) <SOS> hola como estas hoy amigo ? <UNK> <EOS>


In [14]:
class Encoder(nn.Module):
	def __init__(self, hidden_size, emb_dim, vocab_size):
		super().__init__()

		self.hidden_size = hidden_size 
		self.vocab_size = vocab_size

		self.embeddings = nn.Embedding(vocab_size, emb_dim)

		self.Wr = torch.randn(emb_dim, hidden_size)
		self.Ur  = torch.randn(hidden_size, hidden_size)

		# update gate weights
		self.Wz = torch.randn(emb_dim, hidden_size)
		self.Uz  = torch.randn(hidden_size, hidden_size)

		# x-to-h Weights
		self.Wxh = torch.randn(emb_dim, hidden_size)
		self.Whh = torch.randn(hidden_size, hidden_size)

		#
		self.V = torch.randn(hidden_size, hidden_size)

	def forward(self, X: torch.Tensor):
		Xemb = self.embeddings(X)

		T, _ = Xemb.shape
		h = torch.zeros((1, hidden_size))

		for t in range(T):
			r = F.sigmoid((Xemb[t] @ self.Wr) + (h @ self.Ur)) # the reset gate
			z = F.sigmoid((Xemb[t] @ self.Wz) + (h @ self.Uz)) # updated gate
			update = F.tanh(Xemb[t] @ self.Wxh + (r * h) @ self.Whh)
			h = z*h + (1-z)*update

		C = F.tanh(h@self.V)
		return C

In [84]:
class Decoder(nn.Module):
	def __init__(self, hidden_size, emb_dim, vocab_size):
		super().__init__()

		self.hidden_size = hidden_size
		self.vocab_size = vocab_size

		self.embeddings = nn.Embedding(vocab_size, emb_dim)
		#self.V = torch.randn(hidden_size, hidden_size)
		self.V = nn.Linear(hidden_size, hidden_size, bias=False)

		#update gate
		#self.Cz = torch.randn(hidden_size, hidden_size)
		#self.Uz = torch.randn(hidden_size, hidden_size)
		#self.Wz = torch.randn(emb_dim, hidden_size)

		self.Cz = nn.Linear(hidden_size, hidden_size, bias=False)
		self.Uz = nn.Linear(hidden_size, hidden_size, bias=False)
		self.Wz = nn.Linear(emb_dim, hidden_size, bias=False)

		#reset gate
		#self.Cr = torch.randn(hidden_size, hidden_size)
		#self.Ur = torch.randn(hidden_size, hidden_size)
		#self.Wr = torch.randn(emb_dim, hidden_size)

		self.Cr = nn.Linear(hidden_size, hidden_size, bias=False)
		self.Ur = nn.Linear(hidden_size, hidden_size, bias=False)
		self.Wr = nn.Linear(emb_dim, hidden_size, bias=False)

		#update params

		#self.U = torch.randn(hidden_size, hidden_size)
		#self.C = torch.randn(hidden_size, hidden_size)
		#self.W = torch.randn(emb_dim, hidden_size)

		self.U = nn.Linear(hidden_size, hidden_size, bias=False)
		self.C = nn.Linear(hidden_size, hidden_size, bias=False)
		self.W = nn.Linear(emb_dim, hidden_size, bias=False)

		# output
		# Im using 2*vocab_size as the target size here since we are using maxout for output
		# keeping it close to what the paper does oringinally.
		# Might change it to something simpler.
		d = 500

		#self.Oh = torch.randn(hidden_size, 2 * d)
		#self.Oy = torch.randn(emb_dim, 2 * d)
		#self.Oc = torch.randn(hidden_size, 2 * d)

		#self.G = torch.randn(d, vocab_size)

		self.Oh = nn.Linear(hidden_size, 2 * d, bias=False)
		self.Oy = nn.Linear(emb_dim, 2 * d, bias=False)
		self.Oc = nn.Linear(hidden_size, 2 * d, bias=False)

		self.G = nn.Linear(d, vocab_size, bias=False)
		

	def forward(self, c: torch.Tensor, Y: torch.tensor):
		#h =  F.tanh(c @ self.V)
		h = F.tanh(self.V(c))
		Yemb = self.embeddings(Y)
		T, _ = Yemb.shape

		y_logits = []

		for t in range(1, T):
			#update gate
			#z =  F.sigmoid(Yemb[t-1]@self.Wz +  h@self.Uz + c@self.Cz)
			z = F.sigmoid(self.Wz(Yemb[t-1]) + self.Uz(h) + self.Cz(c))
			
			#r =  F.sigmoid(Yemb[t-1]@self.Wr +  h@self.Ur + c@self.Cr)
			#reset gate
			r = F.sigmoid(self.Wr(Yemb[t-1]) + self.Ur(h) + self.Cr(c))

			#candidate update to h
			#update = F.tanh((Yemb[t-1] @ self.W) + r*(h @ self.U + c@self.C))
			update = F.tanh((self.W(Yemb[t-1])) + r * (self.U(h) + self.C(c)))

			#update
			h = z*h + (1-z)*update

			# [1, 2*h] => s = [s1, s2, s3, s4, ..... ] #
			# [[s1, s2],
			# [s3, s4]

			#s_ = h @ self.Oh + Yemb[t-1] @ self.Oy + c @ self.Oc
			s_ = self.Oh(h) + self.Oy(Yemb[t-1]) + self.Oc(c)
			s = s_.view(-1, 2).max(dim=-1).values

			#logits = s@self.G(s)
			logits = self.G(s)

			#probs = F.softmax(logits, dim=-1)

			y_logits.append(logits)
			#log_probs.append(log_p)

		return torch.vstack(y_logits)
			


In [72]:
X = en_tokenizer.encode("Hello how are you today my friend?")
Y = es_tokenizer.encode("hola como estas hoy amigo?")

In [73]:
from dataclasses import dataclass

@dataclass
class Config:
	hidden_size = 32
	emb_dim = 64 # 500 in paper
	encoder_vocab_size = len(en_tokenizer.vocab)
	decoder_vocab_size = len(es_tokenizer.vocab)

In [74]:
class Model(nn.Module):
	def __init__(self, config: Config):
		super().__init__()

		self.encoder = Encoder(config.hidden_size, config.emb_dim, config.encoder_vocab_size)
		self.decoder = Decoder(config.hidden_size, config.emb_dim, config.decoder_vocab_size)

	def forward(self, X: torch.Tensor, Y: torch.Tensor):
		c = self.encoder(X)
		return self.decoder(c, Y)

In [85]:
model = Model(Config)

In [ ]:
with torch.no_grad():
	model.eval()
	y_logits = model.forward(X, Y)
	y_log_probs = F.log_softmax(y_logits, dim=-1)
	L = F.nll_loss(y_log_probs, Y[1:])
	print(f"Initial Loss : " + L.item())
	model.train()

10.803202629089355


In [ ]:
-torch.log(torch.tensor(1/Config.decoder_vocab_size))


tensor(10.3840)

In [289]:
s.view(-1, 2)

tensor([[0, 1],
        [2, 3],
        [4, 5],
        [6, 7],
        [8, 9]])

In [290]:
s.view(N//2, 2)

tensor([[0, 1],
        [2, 3],
        [4, 5],
        [6, 7],
        [8, 9]])

In [57]:
l1 = nn.Linear(2, 3, bias=False)
x = torch.randn(1, 2) 
print(l1(x), x @ l1.weight.T)

tensor([[-0.2018,  0.1348,  0.3534]], grad_fn=<MmBackward0>) tensor([[-0.2018,  0.1348,  0.3534]], grad_fn=<MmBackward0>)


In [ ]:


l1()